In [13]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [14]:
df_rm = pd.read_csv('../data/04_GameSpy_FinalDataset.csv', index_col=0)

### Gestion des NaN des rolling means de Studio - Publisher

In [15]:
df_rm["RM_Studio"]=df_rm["RM_Studio"].fillna(df_rm["RM_Publisher"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Publisher_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Publisher_rate"])
df_rm["RM_Studio_reviews"]=df_rm["RM_Studio_reviews"].fillna(df_rm["RM_Publisher_reviews"])

### Gestion des NaN des rolling means de Licence

In [16]:
df_rm["RM_Licence"]=df_rm["RM_Licence"].fillna(df_rm["RM_Studio"])
df_rm["RM_Licence_score"]=df_rm["RM_Licence_score"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Licence_rate"]=df_rm["RM_Licence_rate"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Licence_reviews"]=df_rm["RM_Licence_reviews"].fillna(df_rm["RM_Studio_reviews"])

### Gestion des NaN des rolling means de Studio - Licence | Publisher - Studio

In [17]:
df_rm["RM_Studio"]=df_rm["RM_Studio"].fillna(df_rm["RM_Licence"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Licence_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Licence_rate"])
df_rm["RM_Studio_reviews"]=df_rm["RM_Studio_reviews"].fillna(df_rm["RM_Licence_reviews"])

df_rm["RM_Publisher"]=df_rm["RM_Publisher"].fillna(df_rm["RM_Studio"])
df_rm["RM_Publisher_score"]=df_rm["RM_Publisher_score"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Publisher_rate"]=df_rm["RM_Publisher_rate"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Publisher_reviews"]=df_rm["RM_Publisher_reviews"].fillna(df_rm["RM_Studio_reviews"])

### Gestion des NaN des rolling means de Publisher Score - Rate | Rate - Score

In [18]:
df_rm["RM_Publisher_score"]=df_rm["RM_Publisher_score"].fillna(df_rm["RM_Publisher_rate"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Licence_score"]=df_rm["RM_Licence_score"].fillna(df_rm["RM_Licence_rate"])

df_rm["RM_Publisher_rate"]=df_rm["RM_Publisher_rate"].fillna(df_rm["RM_Publisher_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Licence_rate"]=df_rm["RM_Licence_rate"].fillna(df_rm["RM_Licence_score"])

### Gestion des NaN de Licence - Name | Studio - Licence | Publisher - Studio

In [19]:
df_rm["GK_licence"]=df_rm["GK_licence"].fillna(df_rm["Name"])
df_rm['Studio']=df_rm['Studio'].fillna(df_rm['GK_licence'])
df_rm['Publisher']=df_rm['Publisher'].fillna(df_rm['Studio'])

# MODELISATION - Regression

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor

In [23]:
df_rm_tosplit = df_rm.drop(['Name', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'], axis=1)

In [24]:
def __encode(df):
    cat_cols = df.select_dtypes(include=['object']).columns.to_list()
    for col in cat_cols:
        i = 1
        occurences = df[col].unique()
        for occ in occurences:
            df.loc[df_rm_tosplit[col]==occ, col] = i
            i+=1

    df[cat_cols] = df[cat_cols].astype('float')

### Echantillonnage 

In [25]:
target=df_rm_tosplit["Global_Sales"]
data=df_rm_tosplit.drop("Global_Sales",axis=1)

X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=40)

### Train - Remplacement des NaN par la moyenne des Rolling Means avant la date de sortie

### Publishers

In [26]:
rm_publisher=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Publisher"]):
    if str(rm)=="nan":
        rm_publisher.append(X_train["RM_Publisher"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_publisher.append(rm)

rm_studio=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Studio"]):
    if str(rm)=="nan":
        rm_studio.append(X_train["RM_Studio"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_studio.append(rm)
        
rm_licence=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Licence"]):
    if str(rm)=="nan":
        rm_licence.append(X_train["RM_Licence"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_licence.append(rm)
        
X_train["RM_Publisher"]=rm_publisher
X_train["RM_Studio"]=rm_studio
X_train["RM_Licence"]=rm_licence

### Publisher , Studio , Licence => Score

In [27]:
rm_publisher_score=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Publisher_score"]):
    if str(rm)=="nan":
        rm_publisher_score.append(X_train["RM_Publisher_score"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_publisher_score.append(rm)
        
rm_studio_score=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Studio_score"]):
    if str(rm)=="nan":
        rm_studio_score.append(X_train["RM_Studio_score"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_studio_score.append(rm)
        
rm_licence_score=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Licence_score"]):
    if str(rm)=="nan":
        rm_licence_score.append(X_train["RM_Licence_score"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_licence_score.append(rm)
        
X_train["RM_Publisher_score"]=rm_publisher_score
X_train["RM_Studio_score"]=rm_studio_score
X_train["RM_Licence_score"]=rm_licence_score

### Publisher , Studio , Licence => Rate

In [28]:
rm_publisher_rate=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Publisher_rate"]):
    if str(rm)=="nan":
        rm_publisher_rate.append(X_train["RM_Publisher_rate"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_publisher_rate.append(rm)
        
rm_studio_rate=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Studio_rate"]):
    if str(rm)=="nan":
        rm_studio_rate.append(X_train["RM_Studio_rate"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_studio_rate.append(rm)
                
rm_licence_rate=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Licence_rate"]):
    if str(rm)=="nan":
        rm_licence_rate.append(X_train["RM_Licence_rate"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_licence_rate.append(rm)
                
X_train["RM_Publisher_rate"]=rm_publisher_rate
X_train["RM_Studio_rate"]=rm_studio_rate
X_train["RM_Licence_rate"]=rm_licence_rate

### Publishers , Studio , Licence => Reviews

In [29]:
rm_publisher_reviews=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Publisher_reviews"]):
    if str(rm)=="nan":
        rm_publisher_reviews.append(X_train["RM_Publisher_reviews"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_publisher_reviews.append(rm)
        
rm_studio_reviews=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Studio_reviews"]):
    if str(rm)=="nan":
        rm_studio_reviews.append(X_train["RM_Studio_reviews"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_studio_reviews.append(rm)
        
rm_licence_reviews=[]
for date,rm in zip(X_train["Date_Sortie"],X_train["RM_Licence_reviews"]):
    if str(rm)=="nan":
        rm_licence_reviews.append(X_train["RM_Licence_reviews"][X_train["Date_Sortie"]<str(date)].mean())
    else:
        rm_licence_reviews.append(rm)
        
X_train["RM_Publisher_reviews"]=rm_publisher_reviews
X_train["RM_Studio_reviews"]=rm_studio_reviews
X_train["RM_Licence_reviews"]=rm_licence_reviews

### Test - Remplacement des NaN par la moyenne des Rolling Means avant la date de sortie

### Publishers

In [30]:
rm_publisher=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Publisher"]):
    if str(rm)=="nan":
        rm_publisher.append(X_test["RM_Publisher"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_publisher.append(rm)

rm_studio=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Studio"]):
    if str(rm)=="nan":
        rm_studio.append(X_test["RM_Studio"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_studio.append(rm)
        
rm_licence=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Licence"]):
    if str(rm)=="nan":
        rm_licence.append(X_test["RM_Licence"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_licence.append(rm)
        
X_test["RM_Publisher"]=rm_publisher
X_test["RM_Studio"]=rm_studio
X_test["RM_Licence"]=rm_licence

### Publishers , Studio , Licence => Score

In [31]:
rm_publisher_score=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Publisher_score"]):
    if str(rm)=="nan":
        rm_publisher_score.append(X_test["RM_Publisher_score"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_publisher_score.append(rm)
        
rm_studio_score=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Studio_score"]):
    if str(rm)=="nan":
        rm_studio_score.append(X_test["RM_Studio_score"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_studio_score.append(rm)
        
rm_licence_score=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Licence_score"]):
    if str(rm)=="nan":
        rm_licence_score.append(X_test["RM_Licence_score"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_licence_score.append(rm)
        
X_test["RM_Publisher_score"]=rm_publisher_score
X_test["RM_Studio_score"]=rm_studio_score
X_test["RM_Licence_score"]=rm_licence_score

### Publishers Rate

In [32]:
rm_publisher_rate=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Publisher_rate"]):
    if str(rm)=="nan":
        rm_publisher_rate.append(X_test["RM_Publisher_rate"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_publisher_rate.append(rm)
        
rm_studio_rate=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Studio_rate"]):
    if str(rm)=="nan":
        rm_studio_rate.append(X_test["RM_Studio_rate"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_studio_rate.append(rm)
                
rm_licence_rate=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Licence_rate"]):
    if str(rm)=="nan":
        rm_licence_rate.append(X_test["RM_Licence_rate"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_licence_rate.append(rm)
                
X_test["RM_Publisher_rate"]=rm_publisher_rate
X_test["RM_Studio_rate"]=rm_studio_rate
X_test["RM_Licence_rate"]=rm_licence_rate

### Publishers Reviews

In [33]:
rm_publisher_reviews=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Publisher_reviews"]):
    if str(rm)=="nan":
        rm_publisher_reviews.append(X_test["RM_Publisher_reviews"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_publisher_reviews.append(rm)
        
rm_studio_reviews=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Studio_reviews"]):
    if str(rm)=="nan":
        rm_studio_reviews.append(X_test["RM_Studio_reviews"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_studio_reviews.append(rm)
        
rm_licence_reviews=[]
for date,rm in zip(X_test["Date_Sortie"],X_test["RM_Licence_reviews"]):
    if str(rm)=="nan":
        rm_licence_reviews.append(X_test["RM_Licence_reviews"][X_test["Date_Sortie"]<str(date)].mean())
    else:
        rm_licence_reviews.append(rm)
        
X_test["RM_Publisher_reviews"]=rm_publisher_reviews
X_test["RM_Studio_reviews"]=rm_studio_reviews
X_test["RM_Licence_reviews"]=rm_licence_reviews

### Calcul des moyennes mobiles sur l'echantillon d'entrainement

In [34]:
RM_Publisher_mean = X_train['RM_Publisher'].mean()
RM_Publisher_score_mean = X_train['RM_Publisher_score'].mean()
RM_Publisher_rate_mean = X_train['RM_Publisher_rate'].mean()
RM_Publisher_reviews_mean = X_train['RM_Publisher_reviews'].mean()

RM_Studio_mean = X_train['RM_Studio'].mean()
RM_Studio_score_mean = X_train['RM_Studio_score'].mean()
RM_Studio_rate_mean = X_train['RM_Studio_rate'].mean()
RM_Studio_reviews_mean = X_train['RM_Studio_reviews'].mean()

RM_Licence_mean = X_train['RM_Licence'].mean()
RM_Licence_score_mean = X_train['RM_Licence_score'].mean()
RM_Licence_rate_mean = X_train['RM_Licence_rate'].mean()
RM_Licence_reviews_mean = X_train['RM_Licence_reviews'].mean()

### Application des moyennes de Rolling means sur les NaN de l'echantillon d'entraînement

In [35]:
X_train['RM_Publisher'] = X_train['RM_Publisher'].fillna(RM_Publisher_mean)
X_train['RM_Publisher_score'] = X_train['RM_Publisher_score'].fillna(RM_Publisher_score_mean)
X_train['RM_Publisher_rate'] = X_train['RM_Publisher_rate'].fillna(RM_Publisher_rate_mean)
X_train['RM_Publisher_reviews'] = X_train['RM_Publisher_reviews'].fillna(RM_Publisher_reviews_mean)

X_train['RM_Studio'] = X_train['RM_Studio'].fillna(RM_Studio_mean)
X_train['RM_Studio_score'] = X_train['RM_Studio_score'].fillna(RM_Studio_score_mean)
X_train['RM_Studio_rate'] = X_train['RM_Studio_rate'].fillna(RM_Studio_rate_mean)
X_train['RM_Studio_reviews'] = X_train['RM_Studio_reviews'].fillna(RM_Studio_reviews_mean)

X_train['RM_Licence'] = X_train['RM_Licence'].fillna(RM_Licence_mean)
X_train['RM_Licence_score'] = X_train['RM_Licence_score'].fillna(RM_Licence_score_mean)
X_train['RM_Licence_rate'] = X_train['RM_Licence_rate'].fillna(RM_Licence_rate_mean)
X_train['RM_Licence_reviews'] = X_train['RM_Licence_reviews'].fillna(RM_Licence_reviews_mean)

### Application des moyennes de Rolling means sur les NaN de l'echantillon de Test

In [36]:
X_test['RM_Publisher'] = X_test['RM_Publisher'].fillna(RM_Publisher_mean)
X_test['RM_Publisher_score'] = X_test['RM_Publisher_score'].fillna(RM_Publisher_score_mean)
X_test['RM_Publisher_rate'] = X_test['RM_Publisher_rate'].fillna(RM_Publisher_rate_mean)
X_test['RM_Publisher_reviews'] = X_test['RM_Publisher_reviews'].fillna(RM_Publisher_reviews_mean)

X_test['RM_Studio'] = X_test['RM_Studio'].fillna(RM_Studio_mean)
X_test['RM_Studio_score'] = X_test['RM_Studio_score'].fillna(RM_Studio_score_mean)
X_test['RM_Studio_rate'] = X_test['RM_Studio_rate'].fillna(RM_Studio_rate_mean)
X_test['RM_Studio_reviews'] = X_test['RM_Studio_reviews'].fillna(RM_Studio_reviews_mean)

X_test['RM_Licence'] = X_test['RM_Licence'].fillna(RM_Licence_mean)
X_test['RM_Licence_score'] = X_test['RM_Licence_score'].fillna(RM_Licence_score_mean)
X_test['RM_Licence_rate'] = X_test['RM_Licence_rate'].fillna(RM_Licence_rate_mean)
X_test['RM_Licence_reviews'] = X_test['RM_Licence_reviews'].fillna(RM_Licence_reviews_mean)

### Suppression de la Date de sortie

In [38]:
X_train=X_train.drop("Date_Sortie",axis=1)

X_test=X_test.drop("Date_Sortie",axis=1)

### Encodage des données

In [39]:
__encode(X_train)
__encode(X_test)

### Choix du modèle le plus performant

In [40]:
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor

lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred_test=lr.predict(X_test)
print(" lr score :" , lr.score(X_test,y_test))

dt=DecisionTreeRegressor()
dt.fit(X_train,y_train)
y_pred_test=dt.predict(X_test)
print(" dt score :" , dt.score(X_test,y_test))

lm=Lasso(alpha=0.1)
lm.fit(X_train,y_train)
y_pred_test=lm.predict(X_test)
print(" lm score :" , lm.score(X_test,y_test))

svr=SVR()
svr.fit(X_train,y_train)
y_pred_test=svr.predict(X_test)
print(" svr score :" , svr.score(X_test,y_test))



rf=RandomForestRegressor()
rf.fit(X_train,y_train)
y_pred_test=rf.predict(X_test)
print(" rf score :" , rf.score(X_test,y_test))


 lr score : 0.07676634286080508
 dt score : -1.8371242850441543
 lm score : 0.009193045108662146
 svr score : 0.01669102380916354
 rf score : -0.11192406364690921


Les scores obtenus par la regression ne sembles pas être concluants